# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [36]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [39]:
csv_path = "output_data/weather_df.csv"

# Read the CSV into a Pandas DataFrame
vacations_df = pd.read_csv(csv_path)
# vacations_df.drop("Unnamed: 0")
vacations_df.reset_index(drop=True)
del vacations_df["Unnamed: 0"]
vacations_df.head()

,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed (mph),Country,Date
0,Bluff,-46.6000,168.3333,57.79,70,14,15.41,NZ,1659583256
1,Chokurdakh,70.6333,147.9167,48.20,61,100,14.74,RU,1659583257
2,Kapaa,22.0752,-159.3190,84.18,67,0,12.66,US,1659583257
3,Mogadishu,2.0371,45.3438,75.60,82,21,16.40,SO,1659583226
4,Punta Arenas,-53.1500,-70.9167,37.51,93,75,16.11,CL,1659583037


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [40]:
# Confirm types of data
vacations_df.dtypes

City                 object
Lat                 float64
Lon                 float64
Max Temp (F)        float64
Humidity              int64
Cloudiness            int64
Wind Speed (mph)    float64
Country              object
Date                  int64
dtype: object

In [41]:
# Confirm there are no null values for Lat, Lon & Humidity
vacations_df.isnull().sum()

City                0
Lat                 0
Lon                 0
Max Temp (F)        0
Humidity            0
Cloudiness          0
Wind Speed (mph)    0
Country             2
Date                0
dtype: int64

In [42]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [51]:
# Create a humidity Heatmap layer
fig = gmaps.figure(center=(30,31), zoom_level=2)

humidity_layer = gmaps.heatmap_layer(vacations_df[["Lat", "Lon"]], weights=vacations_df["Humidity"],
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [61]:
ideal_vacation = vacations_df.dropna()
ideal_vacation.isnull().sum()

City                0
Lat                 0
Lon                 0
Max Temp (F)        0
Humidity            0
Cloudiness          0
Wind Speed (mph)    0
Country             0
Date                0
dtype: int64

In [70]:
ideal_vacation = ideal_vacation[(ideal_vacation["Wind Speed (mph)"]<10) & \
                                (ideal_vacation["Max Temp (F)"]>=70) & (ideal_vacation["Max Temp (F)"]<85) & \
                                (ideal_vacation.Humidity<50) & (ideal_vacation.Cloudiness==0)]
# Per tutor, Mouad using a "." can replace brackets when calling a column.
ideal_vacation.sort_values(by=["Wind Speed (mph)"])

,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed (mph),Country,Date,Hotel Name
154,Barra do Garças,-15.8900,-52.2567,72.32,38,0,0.96,BR,1659583307,Hotel Colombo
537,Panjakent,39.4952,67.6093,76.24,28,0,1.19,TJ,1659583443,Umarion
378,Baena,37.6167,-4.3225,79.66,41,0,1.54,ES,1659583385,Apartamentos Fuente de la salud
509,Kabalo,-6.0500,26.9167,71.64,44,0,3.71,CD,1659583433,null
533,Vilhena,-12.7406,-60.1458,70.77,46,0,3.91,BR,1659583441,Hotel Mirage
343,Gurupi,-11.7292,-49.0686,76.46,30,0,4.29,BR,1659583372,Di Napoli Plaza Hotel
215,Worland,44.0169,-107.9554,83.55,27,0,6.91,US,1659583327,Comfort Inn Worland Hwy 16 to Yellowstone
310,Fairbanks,64.8378,-147.7164,74.08,42,0,6.91,US,1659583203,Westmark Fairbanks Hotel & Conference Center
98,Jalu,29.0331,21.5482,78.17,46,0,7.49,LY,1659583289,إستراحة صلاح الصادق
140,Awjilah,29.1081,21.2869,77.79,48,0,8.05,LY,1659583302,null


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [71]:
# Create a new df called "hotel_df" and add a column for "Hotel Name" 
hotel_df = ideal_vacation
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed (mph),Country,Date,Hotel Name
98,Jalu,29.0331,21.5482,78.17,46,0,7.49,LY,1659583289,
140,Awjilah,29.1081,21.2869,77.79,48,0,8.05,LY,1659583302,
154,Barra do Garças,-15.8900,-52.2567,72.32,38,0,0.96,BR,1659583307,
215,Worland,44.0169,-107.9554,83.55,27,0,6.91,US,1659583327,
310,Fairbanks,64.8378,-147.7164,74.08,42,0,6.91,US,1659583203,


In [72]:
hotel_sample = hotel_df.head(5)
hotel_sample

,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed (mph),Country,Date,Hotel Name
98,Jalu,29.0331,21.5482,78.17,46,0,7.49,LY,1659583289,
140,Awjilah,29.1081,21.2869,77.79,48,0,8.05,LY,1659583302,
154,Barra do Garças,-15.8900,-52.2567,72.32,38,0,0.96,BR,1659583307,
215,Worland,44.0169,-107.9554,83.55,27,0,6.91,US,1659583327,
310,Fairbanks,64.8378,-147.7164,74.08,42,0,6.91,US,1659583203,


In [73]:
# Iterate over each row using .iterrows to complete a json pull and then append to an empty "hotel_name" list
# Append "null" for cities that do not have hotels matching the criteria
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
search_radius = 5000
place_type = "hotel"
# Test code on a sample to verify it works and to obtain location of hotel name in returned json search
# query_url = f"{url}location=42.6001,-73.9662&radius={search_radius}&keyword={place_type}&key={g_key}"
# hotel_response = requests.get(query_url)
# hotel_json = hotel_response.json()
# hotel_json
hotel_name = []
for index, row in hotel_df.iterrows():
    it_lat = row["Lat"]
    it_lon = row["Lon"]
    city = row["City"]
    query_url = f"{url}location={it_lat},{it_lon}&radius={search_radius}&keyword={place_type}&key={g_key}"
    hotel_response = requests.get(query_url)
    hotel_json = hotel_response.json()
    try:
        hotel_name.append(hotel_json["results"][0]["name"])
        print(f"Hotel search for {city} complete.")
    except:
        hotel_name.append("null")
        print(f"Hotel search for {city} complete. {city} does not have a hotel within 5000 meters of coordinates. \
Skipping...")
        pass

Hotel search for Jalu complete.
Hotel search for Awjilah complete. Awjilah does not have a hotel within 5000 meters of coordinates. Skipping...
Hotel search for Barra do Garças complete.
Hotel search for Worland complete.
Hotel search for Fairbanks complete.
Hotel search for Gurupi complete.
Hotel search for Baena complete.
Hotel search for Kabalo complete. Kabalo does not have a hotel within 5000 meters of coordinates. Skipping...
Hotel search for Vilhena complete.
Hotel search for Panjakent complete.


In [74]:
hotel_df["Hotel Name"] = hotel_name
hotel_df

,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed (mph),Country,Date,Hotel Name
98,Jalu,29.0331,21.5482,78.17,46,0,7.49,LY,1659583289,إستراحة صلاح الصادق
140,Awjilah,29.1081,21.2869,77.79,48,0,8.05,LY,1659583302,null
154,Barra do Garças,-15.8900,-52.2567,72.32,38,0,0.96,BR,1659583307,Hotel Colombo
215,Worland,44.0169,-107.9554,83.55,27,0,6.91,US,1659583327,Comfort Inn Worland Hwy 16 to Yellowstone
310,Fairbanks,64.8378,-147.7164,74.08,42,0,6.91,US,1659583203,Westmark Fairbanks Hotel & Conference Center
343,Gurupi,-11.7292,-49.0686,76.46,30,0,4.29,BR,1659583372,Di Napoli Plaza Hotel
378,Baena,37.6167,-4.3225,79.66,41,0,1.54,ES,1659583385,Apartamentos Fuente de la salud
509,Kabalo,-6.0500,26.9167,71.64,44,0,3.71,CD,1659583433,null
533,Vilhena,-12.7406,-60.1458,70.77,46,0,3.91,BR,1659583441,Hotel Mirage
537,Panjakent,39.4952,67.6093,76.24,28,0,1.19,TJ,1659583443,Umarion


In [75]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [76]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=[f"{hotel}" for hotel in hotel_info])

fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))